In [ ]:
import os

In [ ]:
import cv2
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import uuid
import json

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import ResNet50

In [ ]:
def load_image(x):
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding="utf-8") as f:
        label = json.load(f)

    return [label["keypoints"][:4]]

In [ ]:
train_images = tf.data.Dataset.list_files('dataset/train/images/*.jpg', shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (108, 192)))
train_images = train_images.map(lambda x: x/255)

In [ ]:
test_images = tf.data.Dataset.list_files('dataset/test/images/*.jpg', shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (108, 192)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
val_images = tf.data.Dataset.list_files('dataset/val/images/*.jpg', shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (108, 192)))
val_images = val_images.map(lambda x: x/255)

In [ ]:
train_labels = tf.data.Dataset.list_files("dataset/train/labels/*.json", shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [ ]:
test_labels = tf.data.Dataset.list_files("dataset/test/labels/*.json", shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [ ]:
val_labels = tf.data.Dataset.list_files("dataset/val/labels/*.json", shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(6000)
train = train.batch(8)
train = train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(1500)
test = test.batch(8)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(1500)
val = val.batch(8)
val = val.prefetch(4)

In [ ]:
def build_model():
    input_layer = Input(shape=(108, 192, 3))
    
    backbone = ResNet50(include_top=False, weights="imagenet", input_tensor=input_layer)
    backbone.trainable = True
    
    x = backbone.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    outputs = Dense(4, activation="sigmoid")(x)
    
    poseDetector = Model(input_layer,outputs)
    
    return poseDetector

In [ ]:
poseDetector = build_model()

In [ ]:
lr = 1e-5

In [ ]:
poseDetector.compile(loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(lr))

In [ ]:
hist = poseDetector.fit(train,
        validation_data=val,
        epochs=100
    )

In [ ]:
test_data = test.as_numpy_iterator()

In [ ]:
test_sample = test_data.next()
yhat = poseDetector.predict(test_sample[0])
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4):
    s_img = test_sample[0][idx]
    s_keyponts = yhat[idx]
    s_img = np.copy(s_img)
    keys_coords = list(np.multiply(list(zip(s_keyponts[::2], s_keyponts[1::2])), [192, 108]))
    for i in keys_coords:
        cv2.circle(s_img, tuple(map(int, i)), 2, (255,0,0), -1)
    
    ax[idx].imshow(s_img)

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(20,5))

ax.plot(hist.history["loss"], color='teal', label='loss')
ax.plot(hist.history["val_loss"], color='orange', label='val loss')
ax.title.set_text('Loss')
ax.legend()

plt.show()

In [ ]:
poseDetector.save("exported_models/model.h5")